**Ktrain Hate-speech-CNERG/indic-abusive-allInOne-MuRIL**

In [1]:
pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 4.5 MB/s eta 0:00:000:00:0100:01
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.18.0 which is incompatible.
tensorflow-text 2.17.0 requires tensorflow<2.18,>=2.17.0, but you have tensorflow 2.18.0 which is incompatible.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorf

In [2]:
pip uninstall tensorflow keras ktrain -y

Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: keras 3.5.0
Uninstalling keras-3.5.0:
  Successfully uninstalled keras-3.5.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install tensorflow==2.11 keras==2.11 ktrain transformers==4.28.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 73.8 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.0/110.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 41.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 92.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import tensorflow as tf
import keras
import transformers
print("TensorFlow version:", tf.__version__)
print("Transformers version:", transformers.__version__)
print("Keras version:", keras.__version__)

TensorFlow version: 2.11.0
Transformers version: 4.28.1
Keras version: 2.11.0


In [5]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
import torch
from torch.utils.data import Dataset

In [6]:
# Load datasets
train_df = pd.read_csv('/kaggle/input/ma-tam/AWM_train.csv')
dev_df = pd.read_csv('/kaggle/input/ma-tam/AWM_dev.csv')
# test_without_label = pd.read_csv('/kaggle/input/ma-tam/AWM_test_without_labels.csv')
test_with_label = pd.read_csv('/kaggle/input/malatestt/AWM_test_with_labels (1).csv')

In [7]:
test_with_label

,id,Text,Class
0,1,സൂരജ് നിന്റെ ആര് ആണ് ആള് ഇറക്കി പേടിപ്പിക്കുക ആണോ,Non-Abusive
1,2,എത്ര അലക്കി വെളുപ്പിച്ചാലും നിനക്കു അർഹത ഇല്ലാ...,Abusive
2,3,50 ലക്ഷം കയ്യിൽ വയ്ക്കാൻ ഒരിക്കലും യോഗ്യത ഇല്ല...,Non-Abusive
3,4,"""ബിഗ് ബോസ്സിൽ നിങ്ങളുടെ അഭിനയം എന്തായിരുന്നു,മ...",Abusive
4,5,അത് അങ്ങനെയാ നമ്മുടെ ഉള്ളിൽ നന്മ ഉണ്ടെങ്കിൽ പട...,Non-Abusive
...,...,...,...
624,625,ഞാൻ ചെയ്ത വോട്ട് പാഴായില്ല. വളരെ സന്തോഷം,Non-Abusive
625,626,സ്വന്തം കഴിവ് കൊണ്ടല്ല ജയിച്ചത് എന്നറിയുന്ന ദി...,Abusive
626,627,റോബിൻ കഴിഞ്ഞാൽ ദിൽഷ തന്നെയാണ് ഈ ട്രോഫിക് അ...,Non-Abusive
627,628,റോബിനെ കെട്ടണം. ഇല്ലെങ്കിൽ വോട്ടിട്ട അന്തം ഫാൻ...,Non-Abusive


In [8]:
print(train_df.shape)
print(dev_df.shape)
print(test_with_label.shape)

(2933, 2)
(629, 2)
(629, 3)


In [9]:
train_df.head(10)

,Text,Class
0,നവ്യയുടെ കയ്യിന്ന് കിട്ടിയതാണല്ലോ. ഇവൾക്ക് അതൊ...,Abusive
1,"""ഇവരുടെ പ്രശ്നം അസൂയ ആണ്, സുന്ദരികൾ ആയ നടിമാരോ...",Abusive
2,ചുമ്മാതല്ല ഇവളെ പണ്ട് ലവർ അലക്കി വിട്ടത്...വായ...,Abusive
3,"""ഒരു സിനിമയിൽ ജഗതി മദാമ്മയായിട്ട് വരുന്നുണ്ടല്...",Non-Abusive
4,ഈ വർഷത്തെ ബൂലോക തോൽവി പരാജയം അതിനുള്ള ഓസ്‌ക്കാ...,Abusive
5,ബാക്കിൽ നിക്കുന്ന ചേച്ചി : എന്ത് വെറുപ്പിക്കൽ ...,Abusive
6,പ്രയാഗ ശരിക്കും അമേരിക്കൻ അമ്മായി ലുക്ക് ആയല...,Non-Abusive
7,പ്രയാഗ ആർന്നോ ഞാൻ വിചാരിച്ചു ഏതോ ഇംഗ്ലീഷ് കാരി...,Non-Abusive
8,ശെരിക്കും ഇതിന്റെ പാട്ട് വട്ടാണ് വട്ടാണ് എനിക...,Non-Abusive
9,നല്ല ഒരു നായിയായിരുന്നു പക്ഷേ ഇപ്പോ ആരും അഭിനയ...,Abusive


In [10]:
dev_df.head(10)

,Text,Class
0,ഞാനും എന്റെ ഫാമിലി മോളെ വ്യക്തിത്വവും ഗെയിം കണ...,Non-Abusive
1,സാരമില്ല മോളെ നീ ആരുടെ മുന്നിലും കരയരുത് അത് ...,Non-Abusive
2,ഡോക്ടർക്ക് ഇങ്ങനെ തന്നെ വേണം. ഈ വിവരമില്ലാത്ത...,Abusive
3,"""ട്രോളന്മാർക്കും, യൂട്യൂബ് ചാനൽസ് നും ചാകര ആയി .""",Non-Abusive
4,ഇപ്പോ പറഞ്ഞ നോ നേര്തെ പറഞ്ഞഗിൽ ഇത്തരം മോശം കമന...,Non-Abusive
5,നിന്റെ ചേച്ചിമാർ ഓസിന് വോട്ട് പിടിച്ചപ്പോൾ ഓർക...,Non-Abusive
6,നൈസ് ആയി ഒരു കാരണത്തിനു കാത്ത് നിൽക്കുക ആയിരുന...,Abusive
7,അന്തം ഫാൻസിനു ഇനി പിരിഞ്ഞു പോവാം,Non-Abusive
8,"""ഡോക്ടർ ഇത്രെയും ചീപ്പ്‌ ആയി പോയല്ലോ,,, ഇവന് ന...",Non-Abusive
9,നല്ല രീതിയിൽ തീരേണ്ട ഒരു കാര്യമായിരുന്നു. ഇപ്പ...,Non-Abusive


In [11]:
test_with_label.head(10)

,id,Text,Class
0,1,സൂരജ് നിന്റെ ആര് ആണ് ആള് ഇറക്കി പേടിപ്പിക്കുക ആണോ,Non-Abusive
1,2,എത്ര അലക്കി വെളുപ്പിച്ചാലും നിനക്കു അർഹത ഇല്ലാ...,Abusive
2,3,50 ലക്ഷം കയ്യിൽ വയ്ക്കാൻ ഒരിക്കലും യോഗ്യത ഇല്ല...,Non-Abusive
3,4,"""ബിഗ് ബോസ്സിൽ നിങ്ങളുടെ അഭിനയം എന്തായിരുന്നു,മ...",Abusive
4,5,അത് അങ്ങനെയാ നമ്മുടെ ഉള്ളിൽ നന്മ ഉണ്ടെങ്കിൽ പട...,Non-Abusive
5,6,ഇനി കൂടുതൽ ഒന്നും പറയാതിരിക്കുന്നതായിരിക്കും ന...,Non-Abusive
6,7,നിങ്ങളെ ആരും ഇഷ്ടപ്പെടുന്നില്ലാ നിങ്ങളുടെ കാര്...,Abusive
7,8,ഈ ഞാൻ ഞാൻ ...... ഞാനെന്താണെന്ന് മനസ്സിലായി ......,Abusive
8,9,ഇതിൻ്റെ ട്രോൾ കാണാൻ കേറി വരൂ,Non-Abusive
9,10,പണത്തിന്റെ കാര്യം വന്നപ്പോൾ ദേഷ്യം ഒക്കെ പോയോ,Non-Abusive


In [12]:
import re
from bs4 import BeautifulSoup
def remove_punctuations(text):
    punctuations = '''!()-[]{};:'"“\’,<>./?@#$%^&*_~—॥'''
    return ''.join([char for char in text if char not in punctuations])

# Function to replace unwanted strings (URLs, emojis, HTML tags, etc.)
def replace_strings(text):
    # Remove URLs
    url_pattern = r'https?://\S+|www\.\S+'
    text = re.sub(url_pattern, ' ', text)

    # Remove emojis
    emoji_pattern = re.compile("[" 
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"  
        u"\u2000-\u206F"          # general punctuations
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r' ', text)

    # Remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()

    # Normalize spaces
    text = re.sub(r'\s+', ' ', text)

    return text

# Function for full preprocessing
def preprocessing(text):
    cleaned_text = remove_punctuations(replace_strings(text))
    return cleaned_text.lower()


In [13]:
# Apply preprocessing to all datasets
train_df['cleanText'] = train_df['Text'].apply(lambda x: preprocessing(str(x)))
dev_df['cleanText'] = dev_df['Text'].apply(lambda x: preprocessing(str(x)))
test_with_label['cleanText'] = test_with_label['Text'].apply(lambda x: preprocessing(str(x)))


<ipython-input-12-e98d2e6d74aa>:26: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()
<ipython-input-12-e98d2e6d74aa>:26: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()


In [14]:
# Combine all text for vocabulary analysis
train_corpus = train_df['cleanText'].sum()
dev_corpus = dev_df['cleanText'].sum()
test_corpus = test_with_label['cleanText'].sum()

In [15]:
# Vocabulary and OOV analysis
train_vocab = set(train_corpus.split())
dev_vocab = set(dev_corpus.split())
test_vocab = set(test_corpus.split())

oov_dev = dev_vocab - train_vocab
oov_test = test_vocab - train_vocab

print(f"Number of unique words in training data: {len(train_vocab)}")
print(f"Number of unique words in development data: {len(dev_vocab)}")
print(f"Number of unique words in test data: {len(test_vocab)}")
print(f"OOV words in dev dataset: {len(oov_dev)}")
print(f"OOV words in test dataset: {len(oov_test)}")

Number of unique words in training data: 15675
Number of unique words in development data: 4647
Number of unique words in test data: 4586
OOV words in dev dataset: 2525
OOV words in test dataset: 2496


In [16]:
# Label encoding for train and dev datasets
train_df['enc_label'] = train_df['Class'].replace({'Abusive': 1, 'Non-Abusive': 0})
dev_df['enc_label'] = dev_df['Class'].replace({'Abusive': 1, 'Non-Abusive': 0})

print("Training dataset label counts:")
print(train_df['enc_label'].value_counts())
print("Development dataset label counts:")
print(dev_df['enc_label'].value_counts())

Training dataset label counts:
enc_label
1    1531
0    1402
Name: count, dtype: int64
Development dataset label counts:
enc_label
0    326
1    303
Name: count, dtype: int64


<ipython-input-16-7f70e88776ca>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df['enc_label'] = train_df['Class'].replace({'Abusive': 1, 'Non-Abusive': 0})
<ipython-input-16-7f70e88776ca>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dev_df['enc_label'] = dev_df['Class'].replace({'Abusive': 1, 'Non-Abusive': 0})


In [17]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [18]:
# Import necessary libraries
import ktrain
from ktrain import text

In [19]:
categories=['Abusive','Non-Abusive']

In [20]:
import numpy as np
import tensorflow as tf
# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [21]:
# Training data
X_train = train_df['cleanText']  # Preprocessed text column
y_train = train_df['Class']  # Labels for training

# Validation data
X_valid = dev_df['cleanText']  # Preprocessed text column
y_valid = dev_df['Class']  # Labels for validation

# Test data (without labels)
X_test = test_with_label['cleanText']  # Preprocessed text column


In [22]:
# Convert text and labels to lists for ktrain preprocessing
X_train_list = X_train.tolist()
y_train_list = y_train.tolist()
X_valid_list = X_valid.tolist()
y_valid_list = y_valid.tolist()
X_test_list = X_test.tolist()

In [23]:
model_name = 'Hate-speech-CNERG/indic-abusive-allInOne-MuRIL'
trans = text.Transformer(model_name,maxlen=128,class_names=categories)
print("Preprocessing train data...")
train = trans.preprocess_train(X_train_list, y_train_list)

print("Preprocessing validation data...")
valid = trans.preprocess_test(X_valid_list, y_valid_list)

print("Preprocessing test data...")
test = trans.preprocess_test(X_test_list)  # No labels for test data

model = trans.get_classifier()
learner = ktrain.get_learner(model, train_data=train,val_data=valid, batch_size=16)

config.json:   0%|          | 0.00/709 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/950M [00:00<?, ?B/s]

Preprocessing train data...
preprocessing train...
language: ml
train sequence lengths:
	mean : 15
	95percentile : 40
	99percentile : 69


/usr/local/lib/python3.10/dist-packages/ktrain/utils.py:744: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/525 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Is Multi-Label? False
Preprocessing validation data...
preprocessing test...
language: ml
test sequence lengths:
	mean : 17
	95percentile : 46
	99percentile : 75


Preprocessing test data...
preprocessing test...
language: ml
test sequence lengths:
	mean : 17
	95percentile : 46
	99percentile : 79


In [24]:
learner.fit_onecycle(1e-5,8)  



begin training using onecycle policy with max lr of 1e-05...
Epoch 1/8
184/184 [==============================] - 1638s 9s/step - loss: 1.1071 - accuracy: 0.4511 - val_loss: 0.7849 - val_accuracy: 0.4293
Epoch 2/8
184/184 [==============================] - 1611s 9s/step - loss: 0.7221 - accuracy: 0.4616 - val_loss: 0.7033 - val_accuracy: 0.4944
Epoch 3/8
184/184 [==============================] - 1605s 9s/step - loss: 0.6930 - accuracy: 0.5274 - val_loss: 0.6834 - val_accuracy: 0.5644
Epoch 4/8
184/184 [==============================] - 1594s 9s/step - loss: 0.6641 - accuracy: 0.6048 - val_loss: 0.5946 - val_accuracy: 0.6916
Epoch 5/8
184/184 [==============================] - 1586s 9s/step - loss: 0.5970 - accuracy: 0.6833 - val_loss: 0.5755 - val_accuracy: 0.6884
Epoch 6/8
184/184 [==============================] - 1602s 9s/step - loss: 0.4953 - accuracy: 0.7661 - val_loss: 0.5601 - val_accuracy: 0.7250
Epoch 7/8
184/184 [==============================] - 1597s 9s/step - loss: 0.39

In [33]:
from sklearn.metrics import classification_report
import numpy as np

# Step 1: Preprocess the test data using the same transformer used for training and validation
test_data = trans.preprocess_test(X_test_list)  # Preprocessed test data (this converts it into a TransformerDataset)

# Step 2: Get predictions for the test data (this returns probabilities)
preds = learner.predict(test_data)

# Step 3: Convert probabilities to class labels (0 or 1 based on max probability)
pred_labels = np.argmax(preds, axis=1)  
# Convert true labels from strings to numeric format (matching predicted labels)
y_test_true = [1 if label == 'Abusive' else 0 for label in y_test_true]

# Generate classification report
report = classification_report(y_test_true, pred_labels, target_names=categories)
print(report)

preprocessing test...
language: ml
test sequence lengths:
	mean : 17
	95percentile : 46
	99percentile : 79


629/629 [==============================] - 124s 197ms/step
              precision    recall  f1-score   support

     Abusive       1.00      0.45      0.62       629
 Non-Abusive       0.00      0.00      0.00         0

    accuracy                           0.45       629
   macro avg       0.50      0.22      0.31       629
weighted avg       1.00      0.45      0.62       629



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
